In [33]:
import numpy as np
import json
import math 

class ReadAnno:
    def __init__(self, json_path, index ,id_num,process_mode="rectangle"):
        self.json_data = json.load(open(json_path))
        self.filename = self.json_data['images'][index]['file_name'].replace(".png", ".jpg").replace(" (1)","g")
        self.width = 640
        self.height = 480
        self.image_id = self.json_data['images'][index]['id']
        self.coordis = []
        self.id_num =id_num
        assert process_mode in ["rectangle", "polygon"]
        if process_mode == "rectangle" and self.id_num <= 222:
            self.process_rectangle_shapes(index=index,id_num=id_num)
  #      elif process_mode == "polygon":
   #         self.process_polygon_shapes()

    def process_rectangle_shapes(self,index,id_num):
        current_id = self.json_data["annotations"][id_num]['image_id']
        
        while current_id == self.image_id :
            
            print('filename',self.filename)
            print('image_id',self.image_id)
            
            bbox_class = self.json_data["categories"][0]['name']
            bbox = self.json_data["annotations"][id_num]['bbox']
            print('bbox',bbox)
            top_x = bbox[0]*640
            top_y = bbox[1]*480
            bbox_width =bbox[2]*640
            bbox_height = bbox[3]*480

            #ratio_w= 640/432
            #ratio_h = 480/288

            xmin = math.ceil(top_x)-5# / ratio_w
            ymin = math.ceil(top_y)-5 #/ ratio_h
            xmax = math.ceil((top_x + bbox_width))+5# / ratio_w
            ymax = math.ceil((top_y+bbox_height))+5 #/ ratio_h

            self.coordis.append([xmin, ymin, xmax, ymax, bbox_class])
            print('self.coordis',self.coordis)
            
            print('id_num_updated',id_num+1)
            if index <= 168 and id_num < 222:
                id_num = id_num+1
                print('id_num',id_num+1)
            if id_num < 222:
                current_id = self.json_data["annotations"][id_num]['image_id']
                print('update_current_id',current_id)
                self.id_num = id_num
            if id_num >= 222:
                return 

   # def process_polygon_shapes(self):  #### not use here, the code is not suitable for json file from Azure custom ai 
    #    for single_shape in self.json_data['shapes']:
     #       bbox_class = single_shape['label']
      #      temp_points = []
       #     for couple_point in single_shape['points']:
        #        x = float(couple_point[0])
          #      y = float(couple_point[1])
         #       temp_points.append([x, y])
           # temp_points = np.array(temp_points)
           # xmin, ymin = temp_points.min(axis=0)
           # xmax, ymax = temp_points.max(axis=0)
           # self.coordis.append([xmin, ymin, xmax, ymax, bbox_class])

    def get_width_height(self):
        return self.width, self.height

    def get_filename(self):
        return self.filename

    def get_coordis(self):
        return self.coordis,self.id_num

In [34]:
from xml.dom.minidom import Document


class CreateAnno:
    def __init__(self, ):
        self.doc = Document()  # 创建DOM文档对象
        self.anno = self.doc.createElement('annotation')  # 创建根元素
        self.doc.appendChild(self.anno)

        self.add_folder()
        self.add_path()
        self.add_source()
        self.add_segmented()

        # self.add_filename()
        # self.add_pic_size(width_text_str=str(width), height_text_str=str(height), depth_text_str=str(depth))

    def add_folder(self, floder_text_str='JPEGImages'):
        floder = self.doc.createElement('floder')  ##建立自己的开头
        floder_text = self.doc.createTextNode(floder_text_str)  ##建立自己的文本信息
        floder.appendChild(floder_text)  ##自己的内容
        self.anno.appendChild(floder)

    def add_filename(self, filename_text_str='00000.jpg'):
        filename = self.doc.createElement('filename')
        filename_text = self.doc.createTextNode(filename_text_str)
        filename.appendChild(filename_text)
        self.anno.appendChild(filename)

    def add_path(self, path_text_str="None"):
        path = self.doc.createElement('path')
        path_text = self.doc.createTextNode(path_text_str)
        path.appendChild(path_text)
        self.anno.appendChild(path)

    def add_source(self, database_text_str="Unknow"):
        source = self.doc.createElement('source')
        database = self.doc.createElement('database')
        database_text = self.doc.createTextNode(database_text_str)  # 元素内容写入
        database.appendChild(database_text)
        source.appendChild(database)
        self.anno.appendChild(source)

    def add_pic_size(self, width_text_str="0", height_text_str="0", depth_text_str="3"):
        size = self.doc.createElement('size')
        width = self.doc.createElement('width')
        width_text = self.doc.createTextNode(width_text_str)  # 元素内容写入
        width.appendChild(width_text)
        size.appendChild(width)

        height = self.doc.createElement('height')
        height_text = self.doc.createTextNode(height_text_str)
        height.appendChild(height_text)
        size.appendChild(height)

        depth = self.doc.createElement('depth')
        depth_text = self.doc.createTextNode(depth_text_str)
        depth.appendChild(depth_text)
        size.appendChild(depth)

        self.anno.appendChild(size)

    def add_segmented(self, segmented_text_str="0"):
        segmented = self.doc.createElement('segmented')
        segmented_text = self.doc.createTextNode(segmented_text_str)
        segmented.appendChild(segmented_text)
        self.anno.appendChild(segmented)

    def add_object(self,
                   name_text_str="None",
                   xmin_text_str="0",
                   ymin_text_str="0",
                   xmax_text_str="0",
                   ymax_text_str="0",
                   pose_text_str="Unspecified",
                   truncated_text_str="0",
                   difficult_text_str="0"):
        object = self.doc.createElement('object')
        name = self.doc.createElement('name')
        name_text = self.doc.createTextNode(name_text_str)
        name.appendChild(name_text)
        object.appendChild(name)

        pose = self.doc.createElement('pose')
        pose_text = self.doc.createTextNode(pose_text_str)
        pose.appendChild(pose_text)
        object.appendChild(pose)

        truncated = self.doc.createElement('truncated')
        truncated_text = self.doc.createTextNode(truncated_text_str)
        truncated.appendChild(truncated_text)
        object.appendChild(truncated)

        difficult = self.doc.createElement('difficult')
        difficult_text = self.doc.createTextNode(difficult_text_str)
        difficult.appendChild(difficult_text)
        object.appendChild(difficult)

        bndbox = self.doc.createElement('bndbox')
        xmin = self.doc.createElement('xmin')
        xmin_text = self.doc.createTextNode(xmin_text_str)
        xmin.appendChild(xmin_text)
        bndbox.appendChild(xmin)

        ymin = self.doc.createElement('ymin')
        ymin_text = self.doc.createTextNode(ymin_text_str)
        ymin.appendChild(ymin_text)
        bndbox.appendChild(ymin)

        xmax = self.doc.createElement('xmax')
        xmax_text = self.doc.createTextNode(xmax_text_str)
        xmax.appendChild(xmax_text)
        bndbox.appendChild(xmax)

        ymax = self.doc.createElement('ymax')
        ymax_text = self.doc.createTextNode(ymax_text_str)
        ymax.appendChild(ymax_text)
        bndbox.appendChild(ymax)
        object.appendChild(bndbox)

        self.anno.appendChild(object)

    def get_anno(self):
        return self.anno

    def get_doc(self):
        return self.doc

    def save_doc(self, save_path):
        with open(save_path, "w") as f:
            self.doc.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')

In [35]:
import os
from tqdm import tqdm


def json_transform_xml(json_path, xml_path,index ,id_num , process_mode="rectangle"):
    json_path = json_path

    print('hhhh',index,id_num)
    json_anno = ReadAnno(json_path, index = index,id_num=id_num , process_mode=process_mode)
    width, height = json_anno.get_width_height()
    filename = json_anno.get_filename()
    coordis,id_num = json_anno.get_coordis()

    xml_anno = CreateAnno()
    xml_anno.add_filename(filename)
    xml_anno.add_pic_size(width_text_str=str(width), height_text_str=str(height), depth_text_str=str(3))
    for xmin, ymin, xmax, ymax, label in coordis:
        xml_anno.add_object(name_text_str=str(label),
                            xmin_text_str=str(int(xmin)),
                            ymin_text_str=str(int(ymin)),
                            xmax_text_str=str(int(xmax)),
                            ymax_text_str=str(int(ymax)))

    xml_anno.save_doc(xml_path)

    return id_num



if __name__ == "__main__":
    #读取json文件
   # root_json_dir = os.path.abspath(os.path.dirname(__file__))+os.sep+"json"
    #读取xml文件
    id_num =0 
    for index in range(169):
        
        json_path = r'C:\Users\olihu\Documents\yolo-gunshot\csdnyolo\process_labelme\gunshot.json'
        json_data1 = json.load(open(json_path))
        json_filename = json_data1['images'][index]['file_name'].replace(" (1)","g")

        root_save_xml_dir =r'C:\Users\olihu\Documents\yolo-gunshot\csdnyolo\process_labelme\xml_resize'
        #for json_filename in tqdm(os.listdir(root_json_dir)):
        #json_path = os.path.join(root_json_dir, json_filename)


        save_xml_path = os.path.join(root_save_xml_dir, json_filename.replace(".png", ".xml"))
        id_num = json_transform_xml(json_path, save_xml_path, index = index,id_num = id_num , process_mode="rectangle")

hhhh 0 0
filename gun29g.jpg
image_id 1
bbox [0.6928185096153846, 0.29094516594516595, 0.1990384615384615, 0.5685425685425686]
self.coordis [[439, 135, 576, 418, 'gunshot']]
id_num_updated 1
id_num 2
update_current_id 2
hhhh 1 1
filename gun71.jpg
image_id 2
bbox [0.18993389423076923, 0.13077200577200576, 0.11153846153846153, 0.7301587301587301]
self.coordis [[117, 58, 198, 419, 'gunshot']]
id_num_updated 2
id_num 3
update_current_id 3
hhhh 2 2
filename gun27g.jpg
image_id 3
bbox [0.6851262019230769, 0.15674603174603174, 0.20673076923076916, 0.7056277056277056]
self.coordis [[434, 71, 576, 419, 'gunshot']]
id_num_updated 3
id_num 4
update_current_id 4
hhhh 3 3
filename gun53.jpg
image_id 4
bbox [0.1485877403846154, 0.3804112554112554, 0.09807692307692306, 0.42568542568542567]
self.coordis [[91, 178, 163, 392, 'gunshot']]
id_num_updated 4
id_num 5
update_current_id 5
hhhh 4 4
filename gun45.jpg
image_id 5
bbox [0.21589543269230768, 0.26641414141414144, 0.21730769230769234, 0.58152958152

hhhh 46 57
filename gun67g.jpg
image_id 47
bbox [0.17771652321410336, 0.39417895148782167, 0.18039214688968036, 0.4313724742216223]
self.coordis [[109, 185, 235, 402, 'gunshot']]
id_num_updated 58
id_num 59
update_current_id 48
hhhh 47 58
filename gun124.jpg
image_id 48
bbox [0.6755108173076924, 0.12355699855699856, 0.148076923076923, 0.7272727272727273]
self.coordis [[428, 55, 533, 414, 'gunshot']]
id_num_updated 59
id_num 60
update_current_id 49
hhhh 48 59
filename gun100.jpg
image_id 49
bbox [0.3712671149266909, 0.529825449479434, 0.13041097562607024, 0.3219712275009625]
self.coordis [[233, 250, 327, 414, 'gunshot']]
id_num_updated 60
id_num 61
update_current_id 50
hhhh 49 60
filename gun4.jpg
image_id 50
bbox [0.3771145300950833, 0.126513425544773, 0.18922159343422534, 0.7479820764652817]
self.coordis [[237, 56, 368, 425, 'gunshot']]
id_num_updated 61
id_num 62
update_current_id 51
hhhh 50 61
filename gun28g.jpg
image_id 51
bbox [0.26813251244093844, 0.16776908566598936, 0.13652696

filename gun3.jpg
image_id 90
bbox [0.803461817210306, 0.128307188572905, 0.09341321020069238, 0.7479820080402185]
self.coordis [[510, 57, 580, 426, 'gunshot']]
id_num_updated 110
id_num 111
update_current_id 91
hhhh 90 110
filename gun65.jpg
image_id 91
bbox [0.2408954326923077, 0.12355699855699856, 0.17115384615384613, 0.7417027417027418]
self.coordis [[150, 55, 269, 421, 'gunshot']]
id_num_updated 111
id_num 112
update_current_id 92
hhhh 91 111
filename gun147.jpg
image_id 92
bbox [0.5860616187526755, 0.20949690738497817, 0.11068499839469181, 0.634086217234022]
self.coordis [[371, 96, 451, 410, 'gunshot']]
id_num_updated 112
id_num 113
update_current_id 93
hhhh 92 112
filename gun104.jpg
image_id 93
bbox [0.1937800480769231, 0.12788600288600288, 0.13846153846153844, 0.7344877344877345]
self.coordis [[120, 57, 218, 419, 'gunshot']]
id_num_updated 113
id_num 114
update_current_id 94
hhhh 93 113
filename gun29.jpg
image_id 94
bbox [0.2770205563061858, 0.3392710597608124, 0.130410975626

hhhh 138 172
filename gun28.jpg
image_id 139
bbox [0.6716646634615384, 0.34145021645021645, 0.09615384615384626, 0.47330447330447334]
self.coordis [[425, 159, 497, 397, 'gunshot']]
id_num_updated 173
id_num 174
update_current_id 140
hhhh 139 173
filename gun88.jpg
image_id 140
bbox [0.48229167663973144, 0.12359065635531556, 0.14248377980749588, 0.7098039365282246]
self.coordis [[304, 55, 405, 406, 'gunshot']]
id_num_updated 174
id_num 175
update_current_id 141
hhhh 140 174
filename gun67.jpg
image_id 141
bbox [0.39028816451569515, 0.4368273816301149, 0.10898201765414484, 0.44304934103927274]
self.coordis [[245, 205, 325, 428, 'gunshot']]
id_num_updated 175
id_num 176
update_current_id 142
hhhh 141 175
filename gun42g.jpg
image_id 142
bbox [0.42501872171184973, 0.40992155203370234, 0.17245510809435816, 0.45919286616714544]
self.coordis [[268, 192, 388, 423, 'gunshot']]
id_num_updated 176
id_num 177
update_current_id 143
hhhh 142 176
filename gun32g.jpg
image_id 143
bbox [0.2940563625759

In [ ]:
##################not use
bbox=[0.6928185096153846, 0.29094516594516595, 0.1990384615384615, 0.5685425685425686, 'gunshot']
top_x = bbox[0]*640
top_y = bbox[1]*480
bbox_width =bbox[2]*640
bbox_height = bbox[3]*480
ratio_w= 640/432
ratio_h = 480/288
xmin = top_x / ratio_w
ymin = top_y / ratio_h
xmax = (top_x + bbox_width) / ratio_w
ymax = (top_y+bbox_height) / ratio_h
coordis=[xmin, ymin, xmax, ymax]
print(coordis)

In [123]:
testing
index =0

json_path = r'C:\Users\olihu\Documents\yolo-gunshot\csdnyolo\process_labelme\gunshot.json'
json_data1 = json.load(open(json_path))
json_filename = json_data1['images'][index]['file_name'].replace(" (1)","g")

root_save_xml_dir =r'C:\Users\olihu\Documents\yolo-gunshot\csdnyolo\process_labelme\xml'
#for json_filename in tqdm(os.listdir(root_json_dir)):
#json_path = os.path.join(root_json_dir, json_filename)


save_xml_path = os.path.join(root_save_xml_dir, json_filename.replace(".png", ".xml"))

print(save_xml_path)

C:\Users\olihu\Documents\yolo-gunshot\csdnyolo\process_labelme\xml\gun29 (1).xml


In [25]:
json_data1['images'][0]

{'id': 1,
 'width': 432.0,
 'height': 288.0,
 'file_name': 'gun29 (1).png',
 'coco_url': 'AmlDatastore://png/gun29 (1).png',
 'absolute_url': 'https://elpblobtestdata.blob.core.windows.net/png/gun29 (1).png',
 'date_captured': '2021-05-26T08:57:01.2075397Z'}

In [ ]:
C:\Users\olihu\Documents\yolo-gunshot\csdnyolo\yolo3\VOCdevkit\VOC2007\JPEGImages
    

In [135]:
import os
path = r'C:\Users\olihu\Documents\yolo-gunshot\csdnyolo\yolo3\VOCdevkit\VOC2007\JPEGImages'
files = os.listdir(path)


for file in (files):
    filename = os.path.basename(file)
    filename = filename.replace(" (1)","g")
    os.rename(os.path.join(path, file),os.path.join(path, filename))
    

In [134]:
import os
path = r'C:\Users\olihu\Documents\yolo-gunshot\csdnyolo\yolo3\VOCdevkit\VOC2007\JPEGImages'
files = os.listdir(path)


for file in (files):
    filename = os.path.basename(file)
    
    #os.rename(os.path.join(path, file),os.path.join(path, file + '.csv'))
    print(filename)

gun1 (1).png
gun1.png
gun10 (1).png
gun10.png
gun100 (1).png
gun100.png
gun101.png
gun102 (1).png
gun102.png
gun103.png
gun104.png
gun105.png
gun106.png
gun108.png
gun109.png
gun11 (1).png
gun11.png
gun110.png
gun111.png
gun112.png
gun113.png
gun114.png
gun115.png
gun116.png
gun12.png
gun121.png
gun122.png
gun123.png
gun124.png
gun125.png
gun126.png
gun127.png
gun128.png
gun129.png
gun13 (1).png
gun13.png
gun133.png
gun135.png
gun136.png
gun137.png
gun138.png
gun139.png
gun14.png
gun140.png
gun141.png
gun142.png
gun143.png
gun147.png
gun148.png
gun149.png
gun15.png
gun150.png
gun151.png
gun152.png
gun153.png
gun154.png
gun155.png
gun16 (1).png
gun16.png
gun17.png
gun18.png
gun19.png
gun2 (1).png
gun2.png
gun20 (1).png
gun20.png
gun21 (1).png
gun21.png
gun22 (1).png
gun22.png
gun23 (1).png
gun23.png
gun24 (1).png
gun24.png
gun25 (1).png
gun25.png
gun26 (1).png
gun26.png
gun27 (1).png
gun27.png
gun28 (1).png
gun28.png
gun29 (1).png
gun29.png
gun3 (1).png
gun3.png
gun30 (1).png
gun30.png


In [1]:
from PIL import Image
  
# Opens a image in RGB mode
im = Image.open(r'C:\Users\olihu\Downloads\custom-vision-model\custom-vision-model\cache\resize_image/gun68g.jpg')
im.show()

In [2]:
im1 = im.crop((407, 156, 580, 412))
im1.show()

In [ ]:
import cv2  
img = cv2.imread(r'C:/Users/olihu/Documents/yolo-gunshot/csdnyolo/yolo3/VOCdevkit/VOC2007/JPEGImages/gun40g.jpg',1)
cv2.imshow("img", img)
size = (640,480)
shrink = cv2.resize(img, size, interpolation=cv2.INTER_AREA) 
cv2.imshow("shrink", shrink)
cv2.waitKey(0)  


In [ ]:
import cv2
import os

image_ids = open(r'C:\Users\olihu\Downloads\custom-vision-model\custom-vision-model\python\a_file.txt').read().strip().splitlines()
print(image_ids)

for i in range(len(image_ids)):
    image_path = r'C:\Users\olihu\Downloads\custom-vision-model\custom-vision-model\cache\images\%s.jpg' % (image_ids[i])
    save_to = r'C:\Users\olihu\Downloads\custom-vision-model\custom-vision-model\cache\resize_image\%s.jpg' % (image_ids[i])
    print(image_path)
    img = cv2.imread(image_path)
    size = (640, 480)
    resize_image = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    cv2.imwrite(save_to, resize_image)
    # imgall.show()
    # cv2.imshow('image', img)
    # cv2.waitKey()